<a href="https://colab.research.google.com/github/htang2012/.tmux/blob/master/lotto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предсказываем тиражи лото
Предсказывать будем пытаться тиражи столото "дуэль"
https://www.stoloto.ru/duel
Схема игры простая: надо угадать в 2 сериях по 2 числа. Итого - 4 числа. Если угадываем 1 число в тираже, то окупаем затраты на покупку билета. Все, что угадали свыше - наш доход. Также наши расчеты будем строить на гипотезе, что если число давно не выпадало, то вероятность его выпадения становится выше. Предсказывать будем следующие 3 тиража.

Roadmap:
- импорт данных предыдущих тиражей
- подготовка датафрейма
- создание счетчиков когда выпадал номер
- импорт и предсказание с помощью регрессоров sklearn
- импорт и предсказание с помощью нейросети на keras

# Готовим датафрейм

In [ ]:
import pandas as pd
from google.colab import drive, files
import numpy as np

drive.mount('/content/drive/')
output_directory = "/content/drive/My Drive/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#импортируем csv
df = pd.read_csv ('/content/drive/MyDrive/ml/Gosloto_duel_20210101.csv')

In [ ]:
df.head()

,tiraz,1-number,2-number,3-number,4-number,datetime,jackpot
0,145736,8,17,3,25,2021-01-01 18:57:30,296805.0
1,145735,13,1,22,16,2021-01-01 18:42:30,290115.0
2,145734,9,7,7,10,2021-01-01 18:27:30,285555.0
3,145733,23,1,26,20,2021-01-01 18:12:30,281865.0
4,145732,3,21,6,7,2021-01-01 17:57:30,278025.0


In [ ]:
# для дальнейшей работы нужно отсортировать по возрастанию числа в тиражах
df['min12number'] = df.filter(items=['1-number', '2-number']).min(axis=1)
df['max12number'] = df.filter(items=['1-number', '2-number']).max(axis=1)
df['min34number'] = df.filter(items=['3-number', '4-number']).min(axis=1)
df['max34number'] = df.filter(items=['3-number', '4-number']).max(axis=1)

In [ ]:
# подготавливаем функцию, которая будет считать сколько тиражей прошло после того как выпал номер
def count_last_number(number, column):
  number_column = str(number)+'count_in_'+column
  df[number_column] = 0
  df.loc[number_column] = 0
  for i in range(len(df)-2):
    i=i+1
    if df.loc[i, column] == number:
      df.loc[i, number_column] = 0
    else:
      df.loc[i, number_column] = df.loc[i-1, number_column]+1
  df.drop(labels=number_column, axis=0,inplace=True)
  return



In [ ]:
# подготавливаем функцию, которая будет показывать и считать результат предсказания
def win (prediction):
  n=995
  sum_win = 0
  for i in range (0,3):
    a = np.around(prediction[i][0:2], decimals=0)
    a=set(a)
    b = y_test.loc[n].filter(items=['min12number', 'max12number'])
    b=set(b)
    c = list(set(a) & set(b))

    a1 = np.around(prediction[i][2:4], decimals=0)
    a1=set(a1)
    print('Prediction: ',a, a1)
    b1 = y_test.loc[n].filter(items=['min34number', 'max34number'])
    b1=set(b1)
    print('Real number: ', b,b1)
    c1 = list(set(a1) & set(b1))
    print('Intersection number: ',c,c1)
    print('Count intersection number: ', len(c)+len(c1))
    n = n+1
    sum_win = sum_win + len(c)+len(c1)
  print('Count win number: ', sum_win)
  return


In [ ]:
# запускаем счетчик сколько прошло тиражей
spisok = ['min12number', 'max12number', 'min34number', 'max34number']
for i in range (1, 26+1):
  for n in spisok:
    count_last_number(i, n)

In [ ]:
df.tail(50)

,tiraz,1-number,2-number,3-number,4-number,datetime,jackpot,min12number,max12number,min34number,max34number,1count_in_min12number,1count_in_max12number,1count_in_min34number,1count_in_max34number,2count_in_min12number,2count_in_max12number,2count_in_min34number,2count_in_max34number,3count_in_min12number,3count_in_max12number,3count_in_min34number,3count_in_max34number,4count_in_min12number,4count_in_max12number,4count_in_min34number,4count_in_max34number,5count_in_min12number,5count_in_max12number,5count_in_min34number,5count_in_max34number,6count_in_min12number,6count_in_max12number,6count_in_min34number,6count_in_max34number,7count_in_min12number,7count_in_max12number,7count_in_min34number,7count_in_max34number,8count_in_min12number,...,17count_in_min12number,17count_in_max12number,17count_in_min34number,17count_in_max34number,18count_in_min12number,18count_in_max12number,18count_in_min34number,18count_in_max34number,19count_in_min12number,19count_in_max12number,19count_in_min34number,19count_in_max34number,20count_in_min12number,20count_in_max12number,20count_in_min34number,20count_in_max34number,21count_in_min12number,21count_in_max12number,21count_in_min34number,21count_in_max34number,22count_in_min12number,22count_in_max12number,22count_in_min34number,22count_in_max34number,23count_in_min12number,23count_in_max12number,23count_in_min34number,23count_in_max34number,24count_in_min12number,24count_in_max12number,24count_in_min34number,24count_in_max34number,25count_in_min12number,25count_in_max12number,25count_in_min34number,25count_in_max34number,26count_in_min12number,26count_in_max12number,26count_in_min34number,26count_in_max34number
950,144786,24,19,10,12,2020-12-21 15:42:30,1157595.0,19,24,10,12,11,950,7,950,2,950,18,35,3,69,13,150,22,492,3,18,16,60,28,20,12,51,2,3,10,210,32,171,21,...,7,22,8,15,19,50,30,14,0,44,1,23,131,4,109,17,1,39,90,4,29,18,247,1,8,1,369,13,17,0,33,25,102,5,460,9,950,11,950,8
951,144785,5,10,6,24,2020-12-21 15:27:30,1156185.0,5,10,6,24,12,951,8,951,3,951,19,36,4,70,14,151,23,493,4,19,0,61,29,21,13,52,0,4,11,211,33,172,22,...,8,23,9,16,20,51,31,15,1,45,2,24,132,5,110,18,2,40,91,5,30,19,248,2,9,2,370,14,18,1,34,0,103,6,461,10,951,12,951,9
952,144784,22,7,2,9,2020-12-21 15:12:30,1153395.0,7,22,2,9,13,952,9,952,4,952,0,37,5,71,15,152,24,494,5,20,1,62,30,22,14,53,1,5,0,212,34,173,23,...,9,24,10,17,21,52,32,16,2,46,3,25,133,6,111,19,3,41,92,6,31,0,249,3,10,3,371,15,19,2,35,1,104,7,462,11,952,13,952,10
953,144783,17,21,26,14,2020-12-21 14:57:30,1152690.0,17,21,14,26,14,953,10,953,5,953,1,38,6,72,16,153,25,495,6,21,2,63,31,23,15,54,2,6,1,213,35,174,24,...,0,25,11,18,22,53,33,17,3,47,4,26,134,7,112,20,4,0,93,7,32,1,250,4,11,4,372,16,20,3,36,2,105,8,463,12,953,14,953,0
954,144782,16,23,25,12,2020-12-21 14:42:30,1150245.0,16,23,12,25,15,954,11,954,6,954,2,39,7,73,17,154,26,496,7,22,3,64,32,24,16,55,3,7,2,214,36,175,25,...,1,26,12,19,23,54,34,18,4,48,5,27,135,8,113,21,5,1,94,8,33,2,251,5,12,0,373,17,21,4,37,3,106,9,464,0,954,15,954,1
955,144781,15,19,22,21,2020-12-21 14:27:30,1147395.0,15,19,21,22,16,955,12,955,7,955,3,40,8,74,18,155,27,497,8,23,4,65,33,25,17,56,4,8,3,215,37,176,26,...,2,27,13,20,24,55,35,19,5,0,6,28,136,9,114,22,6,2,0,9,34,3,252,0,13,1,374,18,22,5,38,4,107,10,465,1,955,16,955,2
956,144780,23,9,8,26,2020-12-21 14:12:30,1144740.0,9,23,8,26,17,956,13,956,8,956,4,41,9,75,19,156,28,498,9,24,5,66,34,26,18,57,5,9,4,216,38,177,27,...,3,28,14,21,25,56,36,20,6,1,7,29,137,10,115,23,7,3,1,10,35,4,253,1,14,0,375,19,23,6,39,5,108,11,466,2,956,17,956,0
957,144779,23,17,2,8,2020-12-21 13:57:30,1153410.0,17,23,2,8,18,957,14,957,9,957,0,42,10,76,20,157,29,499,10,25,6,67,35,27,19,58,6,10,5,217,39,178,28,...,0,29,15,22,26,57,37,21,7,2,8,30,138,11,116,24,8,4,2,11,36,5,254,2,15,0,376,20,24,7,40,6,109,12,467,3,957,18,957,1
958,144778,14,4,3,20,2020-12-21 13:42:30,1153320.0,4,14,3,20,19,958,15,958,10,958,1,43,11,77,0,158,0,500,11,26,7,68,36,28,20,59,7,11,6,218,40,179,29,...,1,30,16,23,27,58,38,22,8,3,9,31,139,12,

In [ ]:
# добавляем в каждую строку данные по предыдущим выпавшим номерам
n= 26
for i in range(1,n+1):
  name1n = 'last12min'+str(i)
  name2n = 'last12max'+str(i)
  name3n = 'last34min'+str(i)
  name4n = 'last34max'+str(i)
  df[name1n] = df['min12number'].shift(i)
  df[name2n] = df['max12number'].shift(i)
  df[name3n] = df['min34number'].shift(i)
  df[name4n] = df['max34number'].shift(i)

df.head()

,tiraz,1-number,2-number,3-number,4-number,datetime,jackpot,min12number,max12number,min34number,max34number,1count_in_min12number,1count_in_max12number,1count_in_min34number,1count_in_max34number,2count_in_min12number,2count_in_max12number,2count_in_min34number,2count_in_max34number,3count_in_min12number,3count_in_max12number,3count_in_min34number,3count_in_max34number,4count_in_min12number,4count_in_max12number,4count_in_min34number,4count_in_max34number,5count_in_min12number,5count_in_max12number,5count_in_min34number,5count_in_max34number,6count_in_min12number,6count_in_max12number,6count_in_min34number,6count_in_max34number,7count_in_min12number,7count_in_max12number,7count_in_min34number,7count_in_max34number,8count_in_min12number,...,last12min17,last12max17,last34min17,last34max17,last12min18,last12max18,last34min18,last34max18,last12min19,last12max19,last34min19,last34max19,last12min20,last12max20,last34min20,last34max20,last12min21,last12max21,last34min21,last34max21,last12min22,last12max22,last34min22,last34max22,last12min23,last12max23,last34min23,last34max23,last12min24,last12max24,last34min24,last34max24,last12min25,last12max25,last34min25,last34max25,last12min26,last12max26,last34min26,last34max26
0,145736,8,17,3,25,2021-01-01 18:57:30,296805.0,8,17,3,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,145735,13,1,22,16,2021-01-01 18:42:30,290115.0,1,13,16,22,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,145734,9,7,7,10,2021-01-01 18:27:30,285555.0,7,9,7,10,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,0,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,145733,23,1,26,20,2021-01-01 18:12:30,281865.0,1,23,20,26,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,1,3,1,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,145732,3,21,6,7,2021-01-01 17:57:30,278025.0,3,21,6,7,1,4,4,4,4,4,4,4,0,4,4,4,4,4,4,4,4,4,4,4,4,4,0,4,2,4,2,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.dropna(inplace=True)
df.head()

,tiraz,1-number,2-number,3-number,4-number,datetime,jackpot,min12number,max12number,min34number,max34number,1count_in_min12number,1count_in_max12number,1count_in_min34number,1count_in_max34number,2count_in_min12number,2count_in_max12number,2count_in_min34number,2count_in_max34number,3count_in_min12number,3count_in_max12number,3count_in_min34number,3count_in_max34number,4count_in_min12number,4count_in_max12number,4count_in_min34number,4count_in_max34number,5count_in_min12number,5count_in_max12number,5count_in_min34number,5count_in_max34number,6count_in_min12number,6count_in_max12number,6count_in_min34number,6count_in_max34number,7count_in_min12number,7count_in_max12number,7count_in_min34number,7count_in_max34number,8count_in_min12number,...,last12min17,last12max17,last34min17,last34max17,last12min18,last12max18,last34min18,last34max18,last12min19,last12max19,last34min19,last34max19,last12min20,last12max20,last34min20,last34max20,last12min21,last12max21,last34min21,last34max21,last12min22,last12max22,last34min22,last34max22,last12min23,last12max23,last34min23,last34max23,last12min24,last12max24,last34min24,last34max24,last12min25,last12max25,last34min25,last34max25,last12min26,last12max26,last34min26,last34max26
26,145710,19,23,11,15,2021-01-01 10:27:30,226170.0,19,23,11,15,21,26,16,26,4,26,26,26,1,26,2,26,8,26,8,26,12,21,1,26,3,26,6,2,6,26,24,10,26,...,3.0,20.0,5.0,20.0,14.0,26.0,8.0,11.0,7.0,10.0,9.0,25.0,15.0,23.0,16.0,19.0,1.0,5.0,16.0,24.0,3.0,21.0,6.0,7.0,1.0,23.0,20.0,26.0,7.0,9.0,7.0,10.0,1.0,13.0,16.0,22.0,8.0,17.0,3.0,25.0
27,145709,16,4,17,10,2021-01-01 10:12:30,225780.0,4,16,10,17,22,27,17,27,5,27,27,27,2,27,3,27,0,27,9,27,13,22,2,27,4,27,7,3,7,27,25,11,27,...,16.0,21.0,1.0,9.0,3.0,20.0,5.0,20.0,14.0,26.0,8.0,11.0,7.0,10.0,9.0,25.0,15.0,23.0,16.0,19.0,1.0,5.0,16.0,24.0,3.0,21.0,6.0,7.0,1.0,23.0,20.0,26.0,7.0,9.0,7.0,10.0,1.0,13.0,16.0,22.0
28,145708,15,24,26,25,2021-01-01 09:57:30,225120.0,15,24,25,26,23,28,18,28,6,28,28,28,3,28,4,28,1,28,10,28,14,23,3,28,5,28,8,4,8,28,26,12,28,...,20.0,25.0,9.0,18.0,16.0,21.0,1.0,9.0,3.0,20.0,5.0,20.0,14.0,26.0,8.0,11.0,7.0,10.0,9.0,25.0,15.0,23.0,16.0,19.0,1.0,5.0,16.0,24.0,3.0,21.0,6.0,7.0,1.0,23.0,20.0,26.0,7.0,9.0,7.0,10.0
29,145707,25,15,11,10,2021-01-01 09:42:30,224535.0,15,25,10,11,24,29,19,29,7,29,29,29,4,29,5,29,2,29,11,29,15,24,4,29,6,29,9,5,9,29,27,13,29,...,6.0,26.0,6.0,8.0,20.0,25.0,9.0,18.0,16.0,21.0,1.0,9.0,3.0,20.0,5.0,20.0,14.0,26.0,8.0,11.0,7.0,10.0,9.0,25.0,15.0,23.0,16.0,19.0,1.0,5.0,16.0,24.0,3.0,21.0,6.0,7.0,1.0,23.0,20.0,26.0
30,145706,10,7,17,7,2021-01-01 09:27:30,224310.0,7,10,7,17,25,30,20,30,8,30,30,30,5,30,6,30,3,30,12,30,16,25,5,30,7,30,10,6,0,30,0,14,30,...,12.0,21.0,8.0,14.0,6.0,26.0,6.0,8.0,20.0,25.0,9.0,18.0,16.0,21.0,1.0,9.0,3.0,20.0,5.0,20.0,14.0,26.0,8.0,11.0,7.0,10.0,9.0,25.0,15.0,23.0,16.0,19.0,1.0,5.0,16.0,24.0,3.0,21.0,6.0,7.0


In [ ]:
#  чистим датафрейм
df = df.drop(['1-number','2-number','3-number','4-number','tiraz','datetime','jackpot'], axis = 1)
df

,min12number,max12number,min34number,max34number,1count_in_min12number,1count_in_max12number,1count_in_min34number,1count_in_max34number,2count_in_min12number,2count_in_max12number,2count_in_min34number,2count_in_max34number,3count_in_min12number,3count_in_max12number,3count_in_min34number,3count_in_max34number,4count_in_min12number,4count_in_max12number,4count_in_min34number,4count_in_max34number,5count_in_min12number,5count_in_max12number,5count_in_min34number,5count_in_max34number,6count_in_min12number,6count_in_max12number,6count_in_min34number,6count_in_max34number,7count_in_min12number,7count_in_max12number,7count_in_min34number,7count_in_max34number,8count_in_min12number,8count_in_max12number,8count_in_min34number,8count_in_max34number,9count_in_min12number,9count_in_max12number,9count_in_min34number,9count_in_max34number,...,last12min17,last12max17,last34min17,last34max17,last12min18,last12max18,last34min18,last34max18,last12min19,last12max19,last34min19,last34max19,last12min20,last12max20,last34min20,last34max20,last12min21,last12max21,last34min21,last34max21,last12min22,last12max22,last34min22,last34max22,last12min23,last12max23,last34min23,last34max23,last12min24,last12max24,last34min24,last34max24,last12min25,last12max25,last34min25,last34max25,last12min26,last12max26,last34min26,last34max26
26,19,23,11,15,21,26,16,26,4,26,26,26,1,26,2,26,8,26,8,26,12,21,1,26,3,26,6,2,6,26,24,10,26,26,3,14,26,24,15,16,...,3.0,20.0,5.0,20.0,14.0,26.0,8.0,11.0,7.0,10.0,9.0,25.0,15.0,23.0,16.0,19.0,1.0,5.0,16.0,24.0,3.0,21.0,6.0,7.0,1.0,23.0,20.0,26.0,7.0,9.0,7.0,10.0,1.0,13.0,16.0,22.0,8.0,17.0,3.0,25.0
27,4,16,10,17,22,27,17,27,5,27,27,27,2,27,3,27,0,27,9,27,13,22,2,27,4,27,7,3,7,27,25,11,27,27,4,15,27,25,16,17,...,16.0,21.0,1.0,9.0,3.0,20.0,5.0,20.0,14.0,26.0,8.0,11.0,7.0,10.0,9.0,25.0,15.0,23.0,16.0,19.0,1.0,5.0,16.0,24.0,3.0,21.0,6.0,7.0,1.0,23.0,20.0,26.0,7.0,9.0,7.0,10.0,1.0,13.0,16.0,22.0
28,15,24,25,26,23,28,18,28,6,28,28,28,3,28,4,28,1,28,10,28,14,23,3,28,5,28,8,4,8,28,26,12,28,28,5,16,28,26,17,18,...,20.0,25.0,9.0,18.0,16.0,21.0,1.0,9.0,3.0,20.0,5.0,20.0,14.0,26.0,8.0,11.0,7.0,10.0,9.0,25.0,15.0,23.0,16.0,19.0,1.0,5.0,16.0,24.0,3.0,21.0,6.0,7.0,1.0,23.0,20.0,26.0,7.0,9.0,7.0,10.0
29,15,25,10,11,24,29,19,29,7,29,29,29,4,29,5,29,2,29,11,29,15,24,4,29,6,29,9,5,9,29,27,13,29,29,6,17,29,27,18,19,...,6.0,26.0,6.0,8.0,20.0,25.0,9.0,18.0,16.0,21.0,1.0,9.0,3.0,20.0,5.0,20.0,14.0,26.0,8.0,11.0,7.0,10.0,9.0,25.0,15.0,23.0,16.0,19.0,1.0,5.0,16.0,24.0,3.0,21.0,6.0,7.0,1.0,23.0,20.0,26.0
30,7,10,7,17,25,30,20,30,8,30,30,30,5,30,6,30,3,30,12,30,16,25,5,30,7,30,10,6,0,30,0,14,30,30,7,18,30,28,19,20,...,12.0,21.0,8.0,14.0,6.0,26.0,6.0,8.0,20.0,25.0,9.0,18.0,16.0,21.0,1.0,9.0,3.0,20.0,5.0,20.0,14.0,26.0,8.0,11.0,7.0,10.0,9.0,25.0,15.0,23.0,16.0,19.0,1.0,5.0,16.0,24.0,3.0,21.0,6.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,8,22,4,7,7,995,13,995,4,995,38,80,6,114,12,195,3,537,0,63,44,105,2,65,2,96,3,22,9,255,4,0,0,8,9,38,18,103,51,43,...,13.0,15.0,14.0,15.0,9.0,23.0,7.0,13.0,22.0,26.0,19.0,24.0,4.0,13.0,8.0,20.0,15.0,18.0,17.0,26.0,18.0,25.0,4.0,6.0,8.0,19.0,17.0,21.0,1.0,25.0,1.0,25.0,4.0,13.0,4.0,11.0,2.0,18.0,8.0,11.0
996,6,10,7,15,8,996,14,996,5,996,39,81,7,115,13,196,4,538,1,64,45,106,3,66,0,97,4,23,10,256,0,1,1,9,10,39,19,104,52,44,...,13.0,15.0,8.0,21.0,13.0,15.0,14.0,15.0,9.0,23.0,7.0,13.0,22.0,26.0,19.0,24.0,4.0,13.0,8.0,20.0,15.0,18.0,17.0,26.0,18.0,25.0,4.0,6.0,8.0,19.0,17.0,21.0,1.0,25.0,1.0,25.0,4.0,13.0,4.0,11.0
997,5,9,19,22,9,997,15,997,6,997,40,82,8,116,14,197,5,539,2,65,0,107,4,67,1,98,5,24,11,257,1,2,2,10,11,40,20,0,53,45,...,10.0,18.0,3.0,22.0,13.0,15.0,8.0,21.0,13.0,15.0,14.0,15.0,9.0,23.0,7.0,13.0,22.0,26.0,19.0,24.0,4.0,13.0,8.0,20.0

In [ ]:
# подготавливаем тренировочную выборку
X_train = df.drop(['min12number', 'max12number', 'min34number', 'max34number'], axis=1)[:-5]
y_train = df[:-5]

In [ ]:
# подготоавливаем тестовую выборку
X_test = df.drop(['min12number', 'max12number', 'min34number', 'max34number'], axis=1)[-5:]
y_test = df[-5:]

# Предсказываем с помощью sklearn

In [ ]:
# импортируем разные estimator
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error


In [ ]:
# предсказываем с помощью линейной регрессии
# итоговое количество верно предсказанных чисел Count win number:  1
# мало, нужно больше 3
linreg = LinearRegression()
linreg.fit(X_train,y_train)
prediction = linreg.predict(X_test)
print('mae=', mean_absolute_error (y_test, prediction))
win (prediction)

mae= 0.15733322496196128
Prediction:  {24.0, 13.0} {20.0, 6.0}
Real number:  {8.0, 22.0} {4.0, 7.0}
Intersection number:  [] []
Count intersection number:  0
Prediction:  {19.0, 13.0} {28.0, 7.0}
Real number:  {10.0, 6.0} {15.0, 7.0}
Intersection number:  [] [7.0]
Count intersection number:  1
Prediction:  {10.0, 18.0} {33.0, 13.0}
Real number:  {9.0, 5.0} {19.0, 22.0}
Intersection number:  [] []
Count intersection number:  0
Count win number:  1


In [ ]:
# предсказываем с помощью Random Forest
# итоговое количество верно предсказанных чисел Count win number:  4
# отлично, больше 3, окупили затраты
rfr = RandomForestRegressor (random_state=42, n_estimators=10, criterion='mae', max_depth=50, max_features=50, verbose=1, n_jobs=-1)
rfr.fit(X_train,y_train)
prediction = rfr.predict(X_test)
print('mae=', mean_absolute_error (y_test, prediction))
win (prediction)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


mae= 8.886509433962264
Prediction:  {18.0, 6.0} {16.0, 7.0}
Real number:  {8.0, 22.0} {4.0, 7.0}
Intersection number:  [] [7.0]
Count intersection number:  1
Prediction:  {21.0, 6.0} {22.0, 7.0}
Real number:  {10.0, 6.0} {15.0, 7.0}
Intersection number:  [6.0] [7.0]
Count intersection number:  2
Prediction:  {8.0, 20.0} {10.0, 19.0}
Real number:  {9.0, 5.0} {19.0, 22.0}
Intersection number:  [] [19.0]
Count intersection number:  1
Count win number:  4


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  3.0min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.0s finished


In [ ]:
# предсказываем с помощью нейронной сети sklearn
# итоговое количество верно предсказанных чисел Count win number:  1
# мало, нужно больше 3
mlpr = MLPRegressor (random_state=42, hidden_layer_sizes=(500,600,100,100), max_iter=3000)
mlpr.fit(X_train,y_train)
prediction = mlpr.predict(X_test)
print('mae=', mean_absolute_error (y_test, prediction))
win (prediction)

mae= 10.818885673847742
Prediction:  {16.0, 3.0} {23.0, 7.0}
Real number:  {8.0, 22.0} {4.0, 7.0}
Intersection number:  [] [7.0]
Count intersection number:  1
Prediction:  {17.0, 2.0} {8.0, 24.0}
Real number:  {10.0, 6.0} {15.0, 7.0}
Intersection number:  [] []
Count intersection number:  0
Prediction:  {4.0, 13.0} {20.0, 7.0}
Real number:  {9.0, 5.0} {19.0, 22.0}
Intersection number:  [] []
Count intersection number:  0
Count win number:  1


# Предсказываем с помощью нейросети на Keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# нормализуем данные по тренировочной и тестовой выборке для улучшения предсказания
X_train_norm = X_train
X_test_norm = X_test
# Среднее значение
mean = X_train.mean(axis=0)
# Стандартное отклонение
std = X_train.std(axis=0)
X_train_norm -= mean
X_train_norm /= std
X_test_norm -= mean
X_test_norm /= std

In [ ]:
# строим сеть
model = Sequential()
model.add(Dense(1000, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(3000, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(500, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(200, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(y_train.shape[1]))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 1000)              209000    
_________________________________________________________________
dense_21 (Dense)             (None, 3000)              3003000   
_________________________________________________________________
dense_22 (Dense)             (None, 500)               1500500   
_________________________________________________________________
dense_23 (Dense)             (None, 200)               100200    
_________________________________________________________________
dense_24 (Dense)             (None, 212)               42612     
Total params: 4,855,312
Trainable params: 4,855,312
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train_norm, y_train, epochs=200, batch_size=5, verbose=1)

Epoch 1/200
194/194 [==============================] - 1s 3ms/step - loss: 3416.2496 - mae: 25.9887
Epoch 2/200
194/194 [==============================] - 1s 3ms/step - loss: 814.7553 - mae: 16.1929
Epoch 3/200
194/194 [==============================] - 1s 3ms/step - loss: 691.2829 - mae: 15.3011
Epoch 4/200
194/194 [==============================] - 1s 3ms/step - loss: 415.2004 - mae: 12.8109
Epoch 5/200
194/194 [==============================] - 1s 3ms/step - loss: 316.2680 - mae: 11.5186
Epoch 6/200
194/194 [==============================] - 1s 3ms/step - loss: 310.8839 - mae: 11.2239
Epoch 7/200
194/194 [==============================] - 1s 3ms/step - loss: 244.8529 - mae: 10.3544
Epoch 8/200
194/194 [==============================] - 1s 3ms/step - loss: 226.7183 - mae: 10.0183
Epoch 9/200
194/194 [==============================] - 1s 3ms/step - loss: 284.6006 - mae: 10.4340
Epoch 10/200
194/194 [==============================] - 1s 3ms/step - loss: 225.3157 - mae: 9.7683
Epoch 11/

In [ ]:
# предсказываем с помощью нейронной сети, построенной на keras
# итоговое количество верно предсказанных чисел Count win number:  2
# лучше, чем на sklearn, но все равно мало, нужно больше 3
mse, mae = model.evaluate(X_test_norm, y_test, verbose=0)
print("Средняя квадратичная ошибка:", mse)
print("Средняя абсолютная ошибка:", mae)
prediction = model.predict(X_test_norm)
win (prediction)

Средняя квадратичная ошибка: 688.6359252929688
Средняя абсолютная ошибка: 9.82124137878418
Prediction:  {8.0, 20.0} {24.0, 9.0}
Real number:  {8.0, 22.0} {4.0, 7.0}
Intersection number:  [8.0] []
Count intersection number:  1
Prediction:  {9.0, 21.0} {25.0, 10.0}
Real number:  {10.0, 6.0} {15.0, 7.0}
Intersection number:  [] []
Count intersection number:  0
Prediction:  {9.0, 21.0} {9.0, 25.0}
Real number:  {9.0, 5.0} {19.0, 22.0}
Intersection number:  [9.0] []
Count intersection number:  1
Count win number:  2
